In [ ]:
! pip install -q catboost

In [ ]:
path = '/content/drive/MyDrive/Carte de vulnérabilité du COVID-19 en Afrique du Sud by Nimba Hub 3,000,000 GNF/' # change this path to the location where you put your data

# Imports

In [ ]:
import pickle
import numpy as np 
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures, PowerTransformer, KBinsDiscretizer, Normalizer, QuantileTransformer, FunctionTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV,cross_val_score,KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt 
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
print(check_output(["ls", path]).decode("utf8")) #check the files available in the directory

# Reading Data

In [ ]:
test = pd.read_csv(path+'Data/Test_maskedv2.csv')

In [ ]:
train = pd.read_csv(path+'Data/Train_maskedv2.csv')

In [ ]:
varaiable_descr = pd.read_csv(path+ 'Data/variable_descriptions_v2.csv')
varaiable_descr

# Data Exploration

In [ ]:
# Checking shape
#check the numbers of samples and features
print("The train data size before dropping ward feature is : {} ".format(train.shape))
print("The test data size before dropping ward feature is : {} ".format(test.shape))

#Save the 'Id' column
train_ward = train['ward']
test_ward = test['ward']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("ward", axis = 1, inplace = True)
test.drop("ward", axis = 1, inplace = True)

#check again the data size after dropping the 'ward' variable
print("\nThe train data size after dropping ward feature is : {} ".format(train.shape)) 
print("The test data size after dropping ward feature is : {} ".format(test.shape))

In [ ]:
#checking missing values
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
#Checking duplicates
print("Number of duplicates in train:", train.duplicated().sum())
print("Number of duplicates in test:", test.duplicated().sum())

In [ ]:
#Checking the number of unique values per columns (specially if they don't vary at all)
for col in train.columns:
  if train[col].nunique() <=1:
    print('variable name: ', col, '\n',
          'variable description :', varaiable_descr[varaiable_descr['Column']==col]['Description'].values[0], '\n',
          'number_of_unique_values: ', train[col].nunique(), '\n', 
          'unique values: ', train[col].unique(), '\n')

In [ ]:
#Checking the number of unique values per columns (specially if they don't vary at all)
for col in test.columns:
  if test[col].nunique() <=1:
    print('variable name: ', col, '\n',
          'variable description :', varaiable_descr[varaiable_descr['Column']==col]['Description'].values[0], '\n',
          'number_of_unique_values: ', test[col].nunique(), '\n', 
          'unique values: ', test[col].unique(), '\n')

In [ ]:
# droping invariant features
train= train.drop(['dw_12', 'dw_13', 'lan_13'], axis=1)
test= test.drop(['dw_12', 'dw_13', 'lan_13'],axis=1)

In [ ]:
def outlier_checker(col):
  """
  function taking in argument a culumn name of the train
  dataframe and return its scatterplot with the dependant variable
  """
  fig, ax = plt.subplots()
  ax.scatter(x = train[col], y = train['target_pct_vunerable'])
  plt.ylabel('Pourcentage vulnerable', fontsize=13)
  plt.xlabel(f'{col}', fontsize=13)
  plt.show()

In [ ]:
# Let's visualize these scatters
for col in train.columns:
  outlier_checker(col)

**We choose to keep our outliers**

# Feature engineering

In [ ]:
train['avg_total_num_of_ind_per_households'] = (train['total_individuals']/train['total_households'])*100
test['avg_total_num_of_ind_per_households'] = (test['total_individuals']/test['total_households'])*100

In [ ]:
### Applying Kmeans to almost all features and generating a 'cluster' feature.
train_copy=train.copy()
columns=train_copy.drop(["target_pct_vunerable"],1).columns
train_copy=train_copy[columns]
km=KMeans(7,random_state=42)
km=km.fit(train_copy[columns])
train["cluster"]=km.predict(train[columns])
test["cluster"]=km.predict(test[columns])

In [ ]:
## Dropping wards in the training data that have more than 17500 households + 1 outlier.
train = train[train['total_households']<=17500]
train = train[train.index!=1094]

In [ ]:
## Binned feature on total_households
train['total_householdslessthan5000'] = train['total_households'].apply(lambda x:1 if 2500<x<=5000  else 0)
test['total_householdslessthan5000'] = test['total_households'].apply(lambda x:1 if 2500<x<=5000  else 0)

###### A bunch of feature interactions

In [ ]:
train['Individualsperhouse'] = train['total_individuals'] / train['total_households']
test['Individualsperhouse'] = test['total_individuals'] / test['total_households']

In [ ]:
train['Luxury_01'] = train['car_01']+train['stv_00']+train['psa_01']
train['Luxury_00'] = train['car_00'] +train['stv_01']+train['psa_00']

In [ ]:
test['Luxury_01'] = test['car_01']+test['stv_00']+test['psa_01']
test['Luxury_00'] = test['car_00'] +test['stv_01']+test['psa_00']

In [ ]:
train['NoSchoolAttendace'] = train['psa_01'] + train['psa_02']+ train['psa_03']
test['NoSchoolAttendace'] = test['psa_01'] + test['psa_02']+ test['psa_03']

In [ ]:
train['InformalDwellings'] = train['dw_02'] + train['dw_07'] + train['dw_06']
test['InformalDwellings'] = test['dw_02'] + test['dw_07'] + test['dw_06']

In [ ]:
train['TraditionalVSInformalDwellings'] = np.absolute(train['dw_01'] - train['dw_08'])
test['TraditionalVSInformalDwellings'] = np.absolute(test['dw_01'] - test['dw_08'])

In [ ]:
train['total_households']/=train['total_households'].max()
train['total_individuals']/=train['total_individuals'].max()

test['total_households']/=test['total_households'].max()
test['total_individuals']/=test['total_individuals'].max()

In [ ]:
train['SAOldPeopleSesothoVSSetswana'] = np.absolute(train['lan_06'] - train['lan_07'])

test['SAOldPeopleSesothoVSSetswana'] = np.absolute(test['lan_06'] - test['lan_07'])

###### Target encoding + PCA 

In [ ]:
target_mean = train.groupby(['cluster']).mean()[['target_pct_vunerable']]
for i in list(target_mean.columns):
  target_mean.rename({i:i+"_mean"},axis=1,inplace=True)
train = train.merge(target_mean,how="left",on='cluster')
test = test.merge(target_mean,how="left",on='cluster')


In [ ]:
pca = PCA(random_state=42,n_components=1)
pg_features =  train.filter(regex='lan_.*')
train_pca = pca.fit_transform(pg_features)
train['pca_lan_0'] = train_pca[:,0]

In [ ]:
pg_features =  test.filter(regex='lan_.*')
test_pca = pca.transform(pg_features)
test['pca_lan_0'] = test_pca[:,0]

###### Training.

In [ ]:
target = train['target_pct_vunerable']

In [ ]:
train = train.drop(['psa_00','psa_02','psa_03','psa_04','psa_01','lgt_00','stv_01','car_01','lln_01','target_pct_vunerable'], axis=1)
test = test.drop(['psa_00','psa_02','psa_03','psa_04','psa_01','lgt_00','stv_01','car_01','lln_01'], axis=1)

In [ ]:
X = train
y = target

# RandomizedSearch 

In [ ]:
models = [LinearRegression(), KNeighborsRegressor(),
          LinearSVR(),SVR(),DecisionTreeRegressor(),
          RandomForestRegressor(), GradientBoostingRegressor(),
          XGBRegressor(), LGBMRegressor(),MLPRegressor(),
          CatBoostRegressor(verbose = False), Ridge(), Lasso()]
names = ["LinearRegression","KNN","LinearSVR","SVR",
             "DecisionTree","Random_Forest","GBM","XGBoost","LightGBM","Art.Neural_Network","CatBoost", "Lasso", 'Ridge']

In [ ]:
# Possible hyper parameters
linreg_params= {}

knn_params= {"model__n_neighbors": np.linspace(1,19,10, dtype = int).tolist(),
                 "model__weights": ["uniform","distance"],
                 "model__metric":["euclidean","manhattan"]}
linearsvr_params={"model__C": np.arange(1,51),
                  "model__tol": [0.0001, 0.0005, 0.00007, 0.00009, 0.005, 0.1, 0.001]}

svr_params= {"model__kernel" : ["rbf"],
                 "model__gamma": [0.001, 0.01, 0.1, 1, 5, 10 ,50 ,100],
                 "model__C": [1,10,50,100,200,300,1000]}
dtree_params = {"model__min_samples_split" : range(10,500,20),
                "model__max_depth": range(1,20,2),
                }
rf_params = {"model__max_features": ["log2","Auto","None"],
                "model__min_samples_split":[2,3,5],
                "model__min_samples_leaf":[1,3,5],
                "model__bootstrap":[True,False],
                "model__n_estimators":[50,100,150]}
gbm_params = {"model__learning_rate" : [0.001, 0.01, 0.1, 0.05],
             "model__n_estimators": [100,500,100],
             "model__max_depth": [3,5,10],
             "model__min_samples_split": [2,5,10]}
gbm_params = {"model__learning_rate" : [0.001, 0.01, 0.1, 0.05],
             "model__n_estimators": [100,500,100],
             "model__max_depth": [3,5,10],
             "model__min_samples_split": [2,5,10]}

xgb_params ={
                "model__n_estimators":[100,200,400],
                 "model__max_depth":[1,2,3,4],
                 'model__n_estimators': [50, 100, 200],
                 'model__subsample': [ 0.6, 0.8, 1.0],
                 'model__learning_rate': [0.1,0.2, 0.3, 0.4, 0.5],
                 "model__min_samples_split": [1,2,4,6],
                 "model__random_state": [42],
                #  "model_colsample_bytree":[1.0,0.9,0.8]
             }
lgbm_params = { 'model__max_depth': [-1],
        'model__n_estimators': [100, 500, 1000, 2000],
        'model__subsample': [0.6, 0.8, 1.0],
        'model__num_leaves':[31,45,60],
        'model__learning_rate': [0.1,0.05,0.02,0.01],
        "model__min_child_samples": [10,20,30]}
mlpc_params = {"model__alpha": [0.1, 0.01, 0.02, 0.005, 0.0001,0.00001],
              "model__hidden_layer_sizes": [(10,10,10),
                                     (100,100,100),
                                     (100,100),
                                     (3,5), 
                                     (5, 3)],
              "model__solver" : ["lbfgs","adam","sgd"],
              "model__activation": ["relu","tanh"]}
catb_params =  {'model__depth':[2, 3, 4],
              'model__loss_function': ['RMSE'],
              'model__l2_leaf_reg':np.arange(2,31),
              'model__depth':[3,1,2,6,4,5,7,8,9,10],
              'model__iterations':[250,100,500,1000, 100000, 10000],
              'model__learning_rate':[0.03,0.001,0.01,0.06, 0.05, 0.04, 0.03, 0.1,0.2,0.3],
              'model__border_count':[32,5,10,20,50,100,200]}
lasso_params = {'model__alpha':[0.02, 0.024, 0.025, 0.026, 0.03, 1, 10, 100, 1000]}
ridge_params = {'model__alpha':[0.02, 0.024, 0.025, 0.026, 0.03, 1, 10, 100, 200, 230, 250,265, 270, 275, 290, 300, 500],
                'model__max_iter':  [1, 10, 100, 1000, 500]}
cluster_params = {"clustering__n_clusters":[37, 38, 39,40,41,42,43]}
discretizer_params = {'discretizer__n_bins':[3,4,5,6,7,8,9,10,11,12,13,14,15],
                  'discretizer__strategy' : ['uniform', 'quantile', 'kmeans']}
regressor_params = [linreg_params,knn_params, linearsvr_params,svr_params,dtree_params,rf_params,
                     gbm_params, xgb_params,lgbm_params,mlpc_params,catb_params, lasso_params, ridge_params]               

In [ ]:
# Tuninig by Cross Validation  
cv_result = {}
best_estimators = {}
for name, model,regressor_param in zip(names, models,regressor_params):
    print(name)
    pipe = Pipeline([
                     ('transformer', QuantileTransformer()),                 
                     ('discretizer', KBinsDiscretizer(encode='ordinal', n_bins=8)),
                     ('scaler', RobustScaler()),
                     ('model', model)])
    regressor_param = {**regressor_param, **discretizer_params}
    try:
      reg = RandomizedSearchCV(pipe, regressor_param, cv =5, n_jobs =-1,verbose = 2, n_iter=10, scoring=mean_squared_error)
      reg.fit(X,y)
    except ValueError:
      reg = RandomizedSearchCV(pipe, regressor_param, cv =5, n_jobs =-1,verbose = 2, n_iter=10)
      reg.fit(X,y)
    cv_result[name]=reg.best_score_
    best_estimators[name]=reg.best_estimator_
    print(cv_result)   

# Voting Regression

In [ ]:
#Keeping only the estimators with 0.75 as score in cross validation
first_estimators = [(k,v) for k,v in best_estimators.items() if cv_result[k]>=0.75]

# Checking the number of estimators retained
print(len(first_estimators))

# Fitting the data by vote with the estimators retained
votingR = VotingRegressor(estimators = first_estimators, n_jobs =-1)

votingR = votingR.fit(X, y)

# Saving in pickle file the models with its parameters
with open(path+f'voting_beta.csv.pkl', 'wb') as f:
  pickle.dump(votingR, f)

# making predictions on test data
sub = test

y_pred_grid = votingR.predict(sub)

# making submissions
submission_df = pd.DataFrame({'ward': test_ward, 'target_pct_vunerable ': y_pred_grid}) # Creating a submission file

submission_df.to_csv(path+ 'Submissions/'+'gamma_voting.csv', index = False) #alpha 10 #beta 50 gamma 100

In [ ]:
submission_df["target_pct_vunerable "] = np.clip(submission_df["target_pct_vunerable "], 0.099, 47)
submission_df.to_csv(path+ 'Submissions/'+'gamma_voting_clipped.csv', index = False)

In [ ]:
votingR.score(X, y)

In [ ]:
submission_df.columns

In [ ]:
s1 = pd.read_csv(path+"Submissions/alpha_voting.csv") #n_iter=10
s2 = pd.read_csv(path+"Submissions/beta_voting.csv")  #n_iter=50
submission_df['target_pct_vunerable ']=(s1['target_pct_vunerable ']+s2['target_pct_vunerable '])/2
submission_df.to_csv(path+ 'Submissions/'+'alpha_beta.csv', index = False)